In [314]:
using LinearAlgebra
using PlotlyJS

In [315]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    
    
    
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [430]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    maxiter_ = log2(n * m) * 30 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end

        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, B, V
        end
        
        # Step 2 of main loop
        # Performing Golub Kahan at matrix B22
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1])) ? j-1 : l
                
                if l!=k
                    break
                end
                
            end 
            
            # If l is equal to k after previous scan, then l = 1 and there is no B11 matrix
            l = (l == k) ? 1 : l 
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80, save_results = false)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    if save_results
        t = min(m, n)
        results = zeros(2, t-1, maxiter)
        
        for k = 1:maxiter
            for j = 2:t
                results[:, j-1, k] = copy(B[j-1:j, j])
            end
            svd_golub_reinsh!(B, P_l, P_r, 1)
        end
        
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l, results
        else
            return P_l, B, P_r, results
        end
        
    else
        svd_golub_reinsh!(B, P_l, P_r, maxiter)
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l
        else
            return P_l, B, P_r
        end
    end
    
    
    
    
    
end

svd_num (generic function with 3 methods)

## Initial tests

In [317]:
# Example usage 1:
A = randn(5, 7)

5×7 Matrix{Float64}:
 -0.0752544  -0.27911    -1.03886    …   1.25814   -0.155918  -0.681304
  0.964703   -0.0732772   1.12003        1.30399    0.953374  -1.02104
  1.5511     -0.770983    0.0727102     -0.668782  -0.524629   0.347018
 -0.588667   -0.642898   -0.48042        0.952482   0.958895  -1.2715
 -0.0978774  -0.250033   -0.778696      -1.50934   -1.1028    -1.15865

In [318]:
A

5×7 Matrix{Float64}:
 -0.0752544  -0.27911    -1.03886    …   1.25814   -0.155918  -0.681304
  0.964703   -0.0732772   1.12003        1.30399    0.953374  -1.02104
  1.5511     -0.770983    0.0727102     -0.668782  -0.524629   0.347018
 -0.588667   -0.642898   -0.48042        0.952482   0.958895  -1.2715
 -0.0978774  -0.250033   -0.778696      -1.50934   -1.1028    -1.15865

In [319]:
P_l, B, P_r = bidiagonalize(A, true);

In [320]:
B

5×7 Matrix{Float64}:
  1.92311      1.0278    1.11022e-16  …  -2.77556e-17   2.77556e-17
  0.0          1.50666  -2.67619          1.38778e-17  -2.22045e-16
  2.22045e-16  0.0       1.6127           2.22045e-16   2.77556e-17
  0.0          0.0      -1.11022e-16     -5.55112e-17   0.0
 -1.38778e-17  0.0       2.22045e-16     -2.01556       0.0

In [321]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

2.831068712794149e-15

In [322]:
U, S, V = svd_num(A);

In [323]:
show(stdout, "text/plain", U)

5×5 Matrix{Float64}:
  0.54296    0.641674  -0.480324    0.215382  0.127861
  0.320965   0.238041   0.631852   -0.416372  0.517413
 -0.121856  -0.248494   0.0110903   0.654753  0.703262
  0.598224  -0.677529  -0.316621   -0.263871  0.114918
 -0.479015   0.103906  -0.519308   -0.530957  0.456237

In [324]:
show(stdout, "text/plain", S)

5×7 adjoint(::Matrix{Real}) with eltype Real:
 3.73317   0         0        0         0        0  0
 0        -0.86243   0        0         0        0  0
 0         0        -2.57107  0         0        0  0
 0         0         0        2.08033   0        0  0
 0         0         0        0        -2.00803  0  0

In [325]:
show(stdout, "text/plain", V')

7×7 Matrix{Float64}:
 -0.0604059  -0.0926675  -0.0342393  -0.586938     0.663229    0.371578  -0.253285
 -0.214023   -0.469193    0.201151   -0.187346    -0.55859     0.587881   0.029417
 -0.350092   -0.160482   -0.68609    -0.319554    -0.270094   -0.365821  -0.26846
  0.386959   -0.111525   -0.0491615  -0.542597    -0.0768094  -0.212238   0.700037
 -0.73109     0.400272   -0.0434975   0.00111924   0.106532    0.143696   0.52096
  0.374473    0.546537   -0.48756    -0.012583    -0.228899    0.519426  -0.0315543
 -0.0593435   0.523601    0.495604   -0.47299     -0.324824   -0.216803  -0.31696

In [326]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.014127504397588e-15

In [327]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

1.7961904612489356e-15

In [328]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.3204849836108594e-15

In [329]:
# Example usage 2:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -0.714696   0.0624033   1.12736   …   0.633315   -0.348537  -0.826319
  0.323757  -1.00572    -1.48068       1.38463     0.505266  -0.822299
  0.505125  -0.523177   -0.925007     -0.019165    0.663106   0.73671
 -1.01708   -0.102635   -0.157094     -0.692099    0.598486   0.704839
 -1.33441    0.938576   -0.36463      -0.678628   -0.077903  -0.822358
 -0.589611   0.175663    0.684995  …   0.0990116  -0.10495   -1.1874
  1.05027    0.77366    -0.861943      1.26638     0.517102  -0.704662
 -0.306382  -0.683289    0.515447     -0.82917     0.9206    -0.50818
  0.630765  -0.235222   -1.14458      -0.309931   -0.521669   1.09825

In [330]:
U, S, V = svd_num(A);

In [331]:
show(stdout, "text/plain", U)

9×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.243068   0.350834    0.359939   0.30684    -0.113582    0.361267   -0.0928183   0.615786   -0.250808
  0.539719   0.119581   -0.140639   0.475529   -0.660195   -0.0190835   0.02081    -0.103128    0.0347278
  0.323987  -0.130893    0.565663  -0.115029    0.0284721  -0.29922     0.436994    0.334377    0.389331
 -0.263849  -0.467391    0.5114    -0.0289716  -0.407671   -0.181733   -0.15635    -0.272963   -0.389081
 -0.287382   0.194675   -0.172093   0.065944   -0.139533   -0.756677   -0.375812    0.279937    0.184174
 -0.143407   0.651234    0.243886   0.116666    0.11676    -0.253229    0.371838   -0.479636   -0.190174
  0.491066   0.322504    0.121135  -0.614516   -0.0478815  -0.0513994  -0.366017    0.0714196  -0.344294
 -0.21109   -0.0150168  -0.383377  -0.372526   -0.398695   -0.0615609   0.59101     0.26712    -0.293373
  0.293562  -0.246558   -0.135692   0.362333    0.435755   -0.321631    0.113065    0.19745    -0.598526

In [332]:
show(stdout, "text/plain", S)

9×7 Matrix{Real}:
 -3.52867  0        0         0        0        0         0
  0        3.70578  0         0        0        0         0
  0        0        0.679037  0        0        0         0
  0        0        0         1.23455  0        0         0
  0        0        0         0        2.13411  0         0
  0        0        0         0        0        1.86592   0
  0        0        0         0        0        0        -1.73719
  0        0        0         0        0        0         0
  0        0        0         0        0        0         0

In [333]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.570782    0.153094   0.621465  -0.163205  -0.469446  -0.101524  -0.0850197
 -0.0698164   0.170804   0.21171    0.676304   0.308546  -0.062154  -0.603888
 -0.430378    0.124275   0.137757   0.234698   0.475261   0.372509   0.600529
 -0.44833    -0.501943  -0.217425   0.047052   0.210757  -0.670209   0.0629951
  0.356366    0.330939   0.166373   0.395188  -0.217096  -0.533991   0.497345
  0.369976   -0.246417   0.655986  -0.337547   0.486596  -0.146137   0.00313896
 -0.138674    0.713972  -0.211453  -0.434246   0.360862  -0.302299  -0.126988

In [334]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

3.4539905580120652e-15

In [335]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

3.554202815376226e-15

In [336]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.4282695500914144e-15

In [337]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
  0.972751    0.188744  -0.263698    …  -2.25188   -0.0609383   0.484433
  0.188744   -1.34399    2.24641        -3.70711    0.271103   -2.00879
 -0.263698    2.24641    0.00595172     -1.77605   -0.0996247  -1.20724
  1.49879    -2.46259    0.0674447      -2.47969   -0.276394   -1.33608
 -2.25188    -3.70711   -1.77605         5.20593   -0.507679    3.23764
 -0.0609383   0.271103  -0.0996247   …  -0.507679  -0.765514   -2.29763
  0.484433   -2.00879   -1.20724         3.23764   -2.29763     2.51218

In [338]:
U, S, V = svd_num(A);

In [339]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.174943   0.0640936  -0.593528  -0.24224    0.532369    0.5045      0.128041
  0.321801  -0.736645    0.324592  -0.167777   0.207892   -0.0423211   0.418663
  0.239963   0.175661    0.248107   0.246154  -0.44828     0.695414    0.323815
  0.173254  -0.550087   -0.483247   0.125958  -0.468765    0.106167   -0.432414
 -0.739344  -0.292915    0.240032  -0.231778   0.0560065   0.458272   -0.207568
  0.126958  -0.0921622   0.228999   0.708834   0.499319    0.14855    -0.386155
 -0.464211  -0.159688   -0.370437   0.529667  -0.0145795  -0.129367    0.569463

In [340]:
show(stdout, "text/plain", S)

7×7 Matrix{Real}:
 10.6298  0        0         0         0          0        0
  0       5.61064  0         0         0          0        0
  0       0        3.43662   0         0          0        0
  0       0        0        -2.44344   0          0        0
  0       0        0         0        -0.358545   0        0
  0       0        0         0         0         -1.27887  0
  0       0        0         0         0          0        1.85023

In [341]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.174943    0.321801    0.239963   0.173254  -0.739344    0.126958   -0.464211
 -0.0640936   0.736645   -0.175661   0.550087   0.292915    0.0921622   0.159688
 -0.593528    0.324592    0.248107  -0.483247   0.240032    0.228999   -0.370437
 -0.24224    -0.167777    0.246154   0.125958  -0.231778    0.708834    0.529667
  0.532369    0.207892   -0.44828   -0.468765   0.0560065   0.499319   -0.0145795
  0.5045     -0.0423211   0.695414   0.106167   0.458272    0.14855    -0.129367
  0.128041    0.418663    0.323815  -0.432414  -0.207568   -0.386155    0.569463

In [342]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

6.259709455005749e-15

In [343]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

3.4240546759456175e-15

In [344]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.4179592197352932e-15

## Cubic convergence depiction

### Square 7 x 7

In [345]:
A = randn(7, 7)

7×7 Matrix{Float64}:
 -1.04073    0.053814   -2.92224   …   1.44694   -1.39734   -0.725411
 -0.274919  -0.101573   -1.71808      -0.461289  -0.582377   0.479252
  0.355803  -1.49176    -0.34424       0.502877   0.110579   0.701697
 -1.24524    1.34518     0.98848      -2.15816   -1.57999   -0.481111
  0.869701  -0.0943626   0.18653      -0.899751  -0.419328   0.692226
  1.32653    1.24485    -0.316911  …   0.732615   1.15753   -0.0764355
  0.702917  -0.223941    0.198213      1.08421    1.01186    0.913431

In [346]:
U, S, V, results = svd_num(A, 20, true);

In [347]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

7.21828265551578e-14

In [348]:
function plot_convergence_error(error_vectors, labels)
    iterations = 1:length(error_vectors[1])
    traces = Vector{GenericTrace{Dict{Symbol, Any}}}()
    
    for (i, error_vector) in enumerate(error_vectors)
        trace = scatter(
            x = iterations,
            y = error_vector,
            mode = "lines+markers",
            marker_color = i,
            line_shape = "linear",
            name = labels[i]
        )
        push!(traces, trace)
    end
    
    layout = Layout(
        title = "Value of element above diagonal σ_i",
        xaxis_title = "Iteration",
        yaxis_title = "Error",
        yaxis_type = "log",
        yaxis_tickformat = "e",
        showlegend = true
    )
    
    plot(traces, layout)
end

error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [349]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.94247  -2.36683  -4.06312  -4.38968  -4.38026  -4.35131  -4.3174  -4.28156  -4.2395  -4.20198  -4.1652  -4.10192  -4.09692  -4.09692  -4.09692  -4.09692  -4.09692  -4.09692  -4.09692  -4.09692

In [350]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.15009  3.45858  2.10265  1.95387  1.98919  2.09491  2.28887  2.44763  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438  2.54438

In [351]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.69296  -0.987768  -2.02819  -2.5066  -2.4654  -2.34077  -2.1424  -2.00344  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726  -1.92726

In [352]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.03554  -2.31684  -1.12084  -0.907903  -0.906472  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448  -0.906448

In [353]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.69522  -0.68404  -0.651165  -0.644669  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554  -0.642554

In [354]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 0.151774  0.138712  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177  0.138177

In [355]:
S

7×7 Matrix{Real}:
 -4.48122   0        0         0         0          0         0
  0        -4.09692  0         0         0          0         0
  0         0        2.54438   0         0          0         0
  0         0        0        -1.92726   0          0         0
  0         0        0         0        -0.906448   0         0
  0         0        0         0         0         -0.642554  0
  0         0        0         0         0          0         0.138177

### Rectangular 9 x 7

In [356]:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -0.36701    -1.54798   -0.020379   …  -0.982355   1.40085     0.351149
 -0.0448802  -2.34579    0.164276      -0.158639  -1.04649    -0.297506
 -1.27934     0.485512   1.8194        -0.741982  -1.4488      0.535037
 -1.55612     1.68625    1.34646        0.754955  -1.9261      1.15668
 -0.436255    0.713838  -0.888913      -0.573039  -0.211743   -0.494425
  1.69714    -0.51977    0.335986   …   1.90251    0.0500714  -0.926762
 -0.358702    0.654487   0.611217       1.78654    0.296555   -0.109295
  0.135796    0.684345   0.214138      -0.590913  -0.262595   -2.20961
 -1.28752    -2.00483    0.0285233      0.276021  -1.33579    -1.02396

In [357]:
U, S, V, results = svd_num(A, 20, true);

In [358]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.1735637131916358e-15

In [359]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [360]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.64395  2.37812  4.16478  4.25014  4.24923  4.24652  4.24446  4.24312  4.24237  4.24189  4.2415  4.24128  4.24111  4.24106  4.24105  4.24105  4.24105  4.24105  4.24105  4.24105

In [361]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.0555  1.5608  1.66586  -3.39392  -3.72687  -3.76828  -3.77497  -3.77521  -3.77479  -3.77443  -3.774  -3.77383  -3.77373  -3.77373  -3.77373  -3.77373  -3.77373  -3.77373  -3.77373  -3.77373

In [362]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.25944  -2.88199  1.86095  -1.5259  2.58484  2.62538  -2.6902  -2.74802  -2.98885  -2.98876  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887  -2.9887

In [363]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -3.09385  -2.78661  2.72295  1.64491  -0.887134  -0.84827  0.825585  0.807768  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612  0.742612

In [364]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.73596  2.57857  2.17083  2.09512  2.08108  2.11528  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529  2.11529

In [365]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.91348  1.57384  1.55067  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066  1.55066

In [366]:
S

9×7 Matrix{Real}:
 -4.75164  0         0         0       0         0        0
  0        4.24105   0         0       0         0        0
  0        0        -3.77373   0       0         0        0
  0        0         0        -2.9887  0         0        0
  0        0         0         0       0.742612  0        0
  0        0         0         0       0         2.11529  0
  0        0         0         0       0         0        1.55066
  0        0         0         0       0         0        0
  0        0         0         0       0         0        0

### Rectangular 7 x 9

In [367]:
A = randn(7, 9)

7×9 Matrix{Float64}:
  1.57387    1.13738    1.1374    …  -0.294209   -1.09918    0.394352
  0.358062   0.158178   1.98824       1.7301     -1.52803    0.58104
  1.0284    -1.30114    0.712763     -0.858102    0.981143  -2.16646
 -0.299346   1.03899    0.332801     -0.992551   -0.339055  -1.6756
 -0.356325   0.243716   2.30767      -0.091576   -0.178417   1.28975
  0.192341  -1.38566   -0.78685   …  -0.0432002   0.6258     0.133593
  0.417498   0.646646   0.280309     -1.11082    -0.348757   0.916708

In [368]:
U, S, V, results = svd_num(A, 20, true);

In [369]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

3.59598221185909e-14

In [370]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [371]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.93295  4.85235  4.376  3.89805  3.74697  3.71392  3.70711  3.70636  3.7066  3.707  3.69631  3.69631  3.69631  3.69631  3.69631  3.69631  3.69631  3.69631  3.69631  3.69631

In [372]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.6138  -2.46332  -2.59051  -2.68776  -2.86677  -3.22131  -3.63041  -3.70557  -3.71425  -3.71384  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458  -3.72458

In [373]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.63794  2.0447  2.59511  3.4362  3.4035  3.04646  2.70613  2.65104  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455  2.64455

In [374]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.75975  -2.30279  -2.817  -2.09073  -1.9773  -1.96545  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306  -1.96306

In [375]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -3.59769  1.57148  0.911098  0.88948  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466  0.889466

In [376]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.23518  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466  1.23466

In [377]:
S

7×9 adjoint(::Matrix{Real}) with eltype Real:
 -5.48027  0         0        0         0        0         0        0  0
  0        3.69631   0        0         0        0         0        0  0
  0        0        -3.72458  0         0        0         0        0  0
  0        0         0        2.64455   0        0         0        0  0
  0        0         0        0        -1.96306  0         0        0  0
  0        0         0        0         0        0.889466  0        0  0
  0        0         0        0         0        0         1.23466  0  0

### Badly conditioned square matrix

In [378]:
u = randn(7, 1)
u = u/norm(u)
A = (I - u * u') + 1e-9 * (rand(7, 7) - rand(7, 7))

7×7 Matrix{Float64}:
  0.733872    0.071464   -0.217697   …  -0.312908   -0.0893293  -0.116298
  0.071464    0.98081     0.0584588      0.0840259   0.0239878   0.0312298
 -0.217697    0.0584588   0.82192       -0.255964   -0.0730728  -0.0951336
  0.152933   -0.0410676   0.125102       0.179816    0.0513341   0.066832
 -0.312908    0.0840259  -0.255964       0.63209    -0.105031   -0.136741
 -0.0893293   0.0239878  -0.0730728  …  -0.105031    0.970015   -0.0390369
 -0.116298    0.0312298  -0.0951336     -0.136741   -0.0390369   0.949178

In [379]:
U, S, V, results = svd_num(A, 25, true);

In [380]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.34010513502337e-9

In [381]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [382]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -1.21828e-9  0.101992  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [383]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -0.161738  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [384]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [385]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [386]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [387]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [388]:
diag(S)

7-element Vector{Real}:
  1.687987146613597e-10
 -1.0000000003414529
  0.9999999996048783
  0.9999999995637554
  1.0000000002822327
  1.0000000001509228
  1.000000000317067

In [389]:
U, S, V = svd(A);

In [390]:
show(stdout, "text/plain", opnorm(U * diagm(S) * V' - A, 1)/opnorm(A, 1))

3.1633296914595044e-15

In [391]:
S

7-element Vector{Float64}:
 1.000000000736198
 1.0000000007106347
 1.000000000316699
 0.9999999997822111
 0.9999999995945633
 0.9999999991200019
 1.6879872422932705e-10

## Time measurements

#### Bidiagonalize (without matrices - actually almost)

In [392]:
using BenchmarkTools

In [393]:
for size_m = 50:50:200
    A = randn(size_m, size_m)
    @btime bidiagonalize(A);
end

  711.841 μs (1084 allocations: 2.83 MiB)
  4.449 ms (2534 allocations: 21.31 MiB)
  14.576 ms (3984 allocations: 70.72 MiB)
  34.600 ms (5434 allocations: 166.32 MiB)


#### Bidiagonalize (with matrices)

In [394]:
for size_m = 50:50:200
    A = randn(size_m, size_m)
    @btime bidiagonalize(A, true);
end

  1.750 ms (1886 allocations: 6.84 MiB)
  12.127 ms (4136 allocations: 52.52 MiB)
  40.872 ms (6386 allocations: 175.27 MiB)
  104.286 ms (8636 allocations: 413.17 MiB)


#### From bidiag to diag (Golub Kahan) with matrices

In [418]:
execs = zeros(4, 1);
j = 1;
for size_m = 50:50:200
    A = randn(size_m, size_m)
    P_l, B, P_r = bidiagonalize(A, true);
    exec1 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    P_l, B, P_r = bidiagonalize(A, true);
    exec2 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    P_l, B, P_r = bidiagonalize(A, true);
    exec3 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    execs[j] = (exec1 + exec2 + exec3)/3;
    j = j + 1;
end

# 4 times
# bidiagonalize
# bidiagonalize (with matrices)
# from bidiag to diag (Golub Kahan) without singular vectors
# from bidiag to diag (Golub Kahan) with singular vectors

In [419]:
execs

4×1 Matrix{Float64}:
 0.187062399
 0.8744790026666668
 1.6255494160000001
 2.834247203

In [421]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1)/opnorm(A, 1))

2.6006574336156495e-14

#### Native SVD


In [414]:
execs = zeros(4, 1);
j = 1;
for size_m = 50:50:200
    A = randn(size_m, size_m)
    exec1 = @elapsed svd(A);
    exec2 = @elapsed svd(A);
    exec3 = @elapsed svd(A);
    execs[j] = (exec1 + exec2 + exec3)/3;
    j = j + 1;
end

# 4 times
# bidiagonalize
# bidiagonalize (with matrices)
# from bidiag to diag (Golub Kahan) without singular vectors
# from bidiag to diag (Golub Kahan) with singular vectors

In [415]:
execs

4×1 Matrix{Float64}:
 0.0005442566666666667
 0.0021198616666666665
 0.0055248273333333335
 0.009943249